<a href="https://colab.research.google.com/github/alazaradane/marl-robot-navigation/blob/main/Integration_with_Drone_Environment_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Integration with Drone Environment:

When the drone takes an action, the reward is calculated as follows:

## Updates to PPO Agent
The PPO agent implementation remains largely the same, but now it works with the updated environment that has:

Dynamic Start and Target Positions: The target and start points change for every episode.

Dynamic Obstacles: The shaded areas (obstacle zones) change positions dynamically.

The reward system has also been adjusted:

Positive reward for moving closer to the target.

Large reward for reaching the target.

Negative reward for collisions or straying into shaded areas.


In [ ]:
class DroneEnvironment(py_environment.PyEnvironment):
    def __init__(self, start_point, target_point, orange_center, orange_radius, render=False):
        super(DroneEnvironment, self).__init__()

        self.start_point = start_point
        self.target_point = target_point
        self.orange_center = orange_center
        self.orange_radius = orange_radius
        self.reward_calculator = CustomDroneRewardCalculator(self.start_point, self.target_point, self.orange_center, self.orange_radius)
        self.render = render
        self.reset()

    def step(self, action):
        if self.done:
            return self.reset()

        # Action: rotor control
        throttle = action[0] * 10  # Scale action to match the control range
        yaw = action[1] * 10
        pitch = action[2] * 10
        roll = action[3] * 10

        # Apply control to the drone (simplified control model)
        p.setJointMotorControlArray(
            self.drone, range(4), p.VELOCITY_CONTROL, targetVelocities=[throttle, yaw, pitch, roll]
        )

        # Step simulation
        p.stepSimulation()

        # Get the new observation and check if a collision occurred
        observation = self._get_observation()
        collision_occurred = self._has_collision_occurred()

        # Calculate reward
        reward = self.reward_calculator.calculate_reward(observation[:3], action, collision_occurred)

        # Check if episode is done
        if self.reward_calculator.done:
            return ts.termination(observation, reward)
        else:
            return ts.transition(observation, reward=reward, discount=1.0)
